In [1]:
import pandas as pd
import fiona
import geopandas as gpd
from sqlalchemy import create_engine
import numpy as np
#import dedupe
from dotenv import load_dotenv
import os
import pyArango
import usaddress
from usaddress import tag
from scourgify import normalize_address_record
import re
import json

In [2]:
from pyArango.connection import *
conn = Connection(username="root", password="0505")

In [3]:
load_dotenv()
PG_CONNECT = os.getenv("PG_CONNECT")

In [4]:
# create arangoDB database, or open it if it already exists
try:
    corp_db = conn.createDatabase(name="corp_data")
    # from command line:
    # arangoimport --file '''path to CorpData.csv''' --collection corp_data --create-collection true --type csv --server.database corp_data
    # arangoimport --file '''path to CorpIndividualExport.csv''' --collection corp_individual --create-collection true --type csv --server.database corp_data

except:
    corp_db = conn["corp_data"]

In [6]:
# extracting the values of the l3_assess collection
val_aql = "FOR x IN l3_assess RETURN x"
value_query_result = corp_db.AQLQuery(val_aql,rawResults=True, batchSize = 1000)
col_value = {}
ind_val = 0

for value in value_query_result:
    col_value[ind_val] = value
    ind_val += 1
    
# create dataframe from dictionary of dictionaries
l3 = pd.DataFrame.from_dict(data = col_value, orient = 'index')
l3_arr = l3.to_numpy()

_key                                        28345021
_id                               l3_assess/28345021
_rev                                     _eav8fHe---
PROP_ID                                    287-17-25
LOC_ID                              F_760330_2945596
PropAddr      99 POND AVE 606 P584B, BROOKLINE 02445
UNITS                                            1.0
OWNER1                             RIVENSON HOWARD L
OwnAddr      99 POND AVE, # 606, BROOKLINE, MA 02445
TOTAL_VAL                                     636700
FY                                              2021
LS_DATE                               840412800000.0
LS_PRICE                                    180000.0